# MCMC2.5: Dense and Sparse Matrices

This section is not necessarily for Marcov chain Monte Carlo itself, but the use of Sparse matrices is important to speed up quantum Monte Carlo simulations.

## Dense matrix

To use dense matrices, LinearAlgebra provides wrapper to BLAS/LAPACK.

In [1]:
using LinearAlgebra

I recommend Intel MKL instead of OpenBLAS. You can check whether MKL is used by the following command:

In [2]:
BLAS.vendor()

:mkl

1D Array is called Vector.

In [3]:
Array{Int64, 1} == Vector{Int64}

true

2D Array is called Matrix, which I later call dense matrix.

In [4]:
Array{Float64, 2} == Matrix{Float64}

true

Matrix and Array of Array are different.

In [5]:
matrix = [1 2
                 3 4]
array =[[1, 2], [3, 4]]
matrix == array

false

Julia's support on 3-rank tensors is limited, but still we can define and use them.

In [6]:
tensor = zeros(Float64, 2, 2, 2)

2×2×2 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 2] =
 0.0  0.0
 0.0  0.0

Be careful, matrices are stored columnwise.

In [7]:
mat = ones(10000, 10000)
mat[:, 1]
mat[1, :]
@time mat[:, 1]
@time mat[1, :]
println("End!")

  0.000023 seconds (7 allocations: 78.375 KiB)
  0.000395 seconds (7 allocations: 78.375 KiB)
End!


Thus, vertical vectors are more important.

In [8]:
[1.0 2.0; 3.0 4.0] * [5.0; 6.0]

2-element Array{Float64,1}:
 17.0
 39.0

## BLAS/LAPACK

Dense matrices are not memory-efficient, but support BLAS/LAPACK. The most important BLAS operation in quantum Monte Carlo is rank-1 update (or other finite-rank updates).

In [2]:
A = rand(Float64, 1000, 1000)

1000×1000 Array{Float64,2}:
 0.0309637  0.963237   0.659809  0.161889  …  0.554004  0.24017    0.701533 
 0.252961   0.256455   0.887348  0.720892     0.137021  0.62597    0.516955 
 0.801891   0.0355228  0.659865  0.146784     0.910977  0.957655   0.226707 
 0.302365   0.489994   0.748127  0.362462     0.202818  0.695144   0.524791 
 0.589881   0.720213   0.731793  0.327218     0.316837  0.824122   0.338946 
 0.842267   0.509839   0.580669  0.392607  …  0.280414  0.607666   0.307534 
 0.838772   0.4507     0.245435  0.641608     0.358249  0.907074   0.31072  
 0.666076   0.886652   0.607456  0.702718     0.702585  0.300749   0.108952 
 0.145292   0.694542   0.603646  0.832036     0.33536   0.365673   0.766862 
 0.106796   0.217884   0.750812  0.134405     0.296914  0.294457   0.533726 
 0.0309095  0.778457   0.622044  0.682674  …  0.679265  0.937097   0.186918 
 0.0482619  0.757842   0.958546  0.866743     0.446877  0.0501466  0.808526 
 0.0540594  0.45534    0.566973  0.176375     0.

Let's check the Sherman-Morrison formula!
$$\left( A+\vec{u} \vec{v}^T \right)^{-1} = A^{-1} - \frac{A^{-1} \vec{u} \vec{v}^TA^{-1}}{1 + \vec{v}^T A^{-1} \vec{u}}$$
Of course, the vectors are vertical. I define $B = A+\vec{u} \vec{v}^T$

## Sparse matrix

If your program is intensively using sparse matrices, you should use python instead because Julia only supports CSC matrix. Julia's native support for sparse matrices is not strong, so I do not recommend to write a code using multiple types of sparse matrices in Julia.

## Iterative solvers

~ under construction ~